# Financial Data Analysis Example

This notebook demonstrates how to use the data fetching library for quantitative analysis.

In [ ]:
# Setup
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Import our modules
from src.polygon import StockDataFetcher, OptionsDataFetcher, TreasuryDataFetcher

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Stock Analysis

In [ ]:
# Initialize stock fetcher
stock_fetcher = StockDataFetcher()

# Fetch data for multiple tech stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'NVDA']
end_date = datetime.now()
start_date = end_date - timedelta(days=252)  # 1 year

stock_data = stock_fetcher.fetch_multiple_bars(
    tickers=tickers,
    start_date=start_date,
    end_date=end_date,
    timeframe='day'
)

In [ ]:
# Calculate returns
returns = pd.DataFrame()
for ticker, data in stock_data.items():
    if not data.empty:
        returns[ticker] = data['close'].pct_change()

# Plot cumulative returns
cumulative_returns = (1 + returns).cumprod() - 1

plt.figure(figsize=(12, 6))
for ticker in returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[ticker], label=ticker)

plt.title('Cumulative Returns - Tech Stocks')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
correlation = returns.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Stock Returns Correlation Matrix')
plt.tight_layout()
plt.show()

## 2. Options Analysis

In [ ]:
# Initialize options fetcher
options_fetcher = OptionsDataFetcher()

# Get SPY options chain
spy_price = stock_fetcher.fetch_snapshot('SPY')['last_trade']['price']
print(f"Current SPY price: ${spy_price:.2f}")

# Fetch near-the-money options
options = options_fetcher.fetch_options_chain(
    underlying_ticker='SPY',
    strike_price_gte=spy_price * 0.95,
    strike_price_lte=spy_price * 1.05
)

In [ ]:
# Analyze options by expiration
expirations = {}
for opt in options:
    exp = opt.expiration_date
    if exp not in expirations:
        expirations[exp] = {'calls': [], 'puts': []}
    expirations[exp][f"{opt.contract_type}s"].append(opt)

# Plot strike distribution for nearest expiration
if expirations:
    nearest_exp = min(expirations.keys())
    
    call_strikes = [opt.strike_price for opt in expirations[nearest_exp]['calls']]
    put_strikes = [opt.strike_price for opt in expirations[nearest_exp]['puts']]
    
    plt.figure(figsize=(10, 6))
    plt.hist(call_strikes, bins=20, alpha=0.5, label='Calls', color='green')
    plt.hist(put_strikes, bins=20, alpha=0.5, label='Puts', color='red')
    plt.axvline(spy_price, color='black', linestyle='--', label='Current Price')
    plt.xlabel('Strike Price')
    plt.ylabel('Count')
    plt.title(f'Strike Distribution - SPY Options Expiring {nearest_exp}')
    plt.legend()
    plt.tight_layout()
    plt.show()

## 3. Treasury Yield Analysis

In [ ]:
# Initialize treasury fetcher
treasury_fetcher = TreasuryDataFetcher()

# Fetch current yield curve
yield_curve = treasury_fetcher.fetch_yield_curve(datetime.now())

# Plot yield curve
plt.figure(figsize=(10, 6))
plt.plot(yield_curve['years'], yield_curve['yield'], 'bo-', markersize=8)
plt.xlabel('Maturity (Years)')
plt.ylabel('Yield (%)')
plt.title('Current US Treasury Yield Curve')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Fetch historical yield data
yield_history = treasury_fetcher.fetch_yield_curve_history(
    start_date=start_date,
    end_date=end_date,
    maturities=['2Y', '10Y', '30Y']
)

# Calculate and plot yield spreads
spreads = treasury_fetcher.calculate_yield_spreads(yield_history)

plt.figure(figsize=(12, 6))
for spread in spreads.columns:
    plt.plot(spreads.index, spreads[spread], label=spread)

plt.axhline(0, color='black', linestyle='--', alpha=0.5)
plt.xlabel('Date')
plt.ylabel('Spread (%)')
plt.title('Treasury Yield Spreads Over Time')
plt.legend()
plt.tight_layout()
plt.show()

## 4. Cross-Asset Analysis

In [ ]:
# Compare stock returns with treasury yields
spy_data = stock_fetcher.fetch_bars('SPY', start_date, end_date)
spy_returns = spy_data['close'].pct_change().rolling(20).mean() * 252  # Annualized

# Plot stock returns vs 10Y yield
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(spy_returns.index, spy_returns, 'b-', label='SPY 20-day Returns (Ann.)')
ax1.set_xlabel('Date')
ax1.set_ylabel('SPY Returns (%)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.plot(yield_history.index, yield_history['10Y'], 'r-', label='10Y Treasury Yield')
ax2.set_ylabel('10Y Yield (%)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.title('SPY Returns vs 10Y Treasury Yield')
plt.tight_layout()
plt.show()

## 5. Risk Metrics

In [ ]:
# Calculate various risk metrics
risk_metrics = pd.DataFrame()

for ticker in returns.columns:
    ticker_returns = returns[ticker].dropna()
    
    # Basic metrics
    metrics = {
        'Annual Return': ticker_returns.mean() * 252,
        'Annual Volatility': ticker_returns.std() * np.sqrt(252),
        'Sharpe Ratio': (ticker_returns.mean() * 252) / (ticker_returns.std() * np.sqrt(252)),
        'Max Drawdown': (ticker_returns.cumsum().expanding().max() - ticker_returns.cumsum()).max(),
        'Skewness': ticker_returns.skew(),
        'Kurtosis': ticker_returns.kurtosis()
    }
    
    risk_metrics[ticker] = metrics

# Display risk metrics
risk_metrics = risk_metrics.T
print("Risk Metrics Summary:")
print(risk_metrics.round(3))

In [ ]:
# Visualize risk-return profile
plt.figure(figsize=(10, 8))

for ticker in risk_metrics.index:
    plt.scatter(
        risk_metrics.loc[ticker, 'Annual Volatility'] * 100,
        risk_metrics.loc[ticker, 'Annual Return'] * 100,
        s=200,
        label=ticker
    )
    plt.annotate(ticker, 
                 (risk_metrics.loc[ticker, 'Annual Volatility'] * 100,
                  risk_metrics.loc[ticker, 'Annual Return'] * 100),
                 xytext=(5, 5), textcoords='offset points')

plt.xlabel('Annual Volatility (%)')
plt.ylabel('Annual Return (%)')
plt.title('Risk-Return Profile')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()